In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 200)
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.metrics import f1_score
import gc

# TEAM CONDA

In [2]:
# !pip install category_encoders

In [3]:
train = pd.read_csv('data_train.csv',encoding='ISO-8859-1')
test = pd.read_csv('data_test.csv',encoding='ISO-8859-1')
train.shape, test.shape

/home/villacorta/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (76,77,78,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/villacorta/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (76,77) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


((256215, 126), (44072, 116))

In [4]:
train.columns

Index(['COD_ALUMNO', 'SEMESTRE', 'TIPO_SEMESTRE', 'CARRERA',
       'SITUACION_MATRICULA', 'CURSOSAPROBADOS', 'CURSOSDESAPROBADOS',
       'CURSOSNSP', 'CREDAPROBADOS', 'CREDDESAPROBADOS',
       ...
       'SITUACION_DEUDA_BOLETA_1', 'SITUACION_DEUDA_BOLETA_2',
       'SITUACION_DEUDA_BOLETA_3', 'SITUACION_DEUDA_BOLETA_4',
       'SITUACION_DEUDA_BOLETA_5', 'SITUACION_DEUDA_BOLETA_6',
       'SITUACION_DEUDA_BOLETA_7', 'SITUACION_DEUDA_BOLETA_8',
       'SITUACION_DEUDA_BOLETA_9', 'SITUACION_DEUDA_BOLETA_10'],
      dtype='object', length=126)

In [5]:
test.columns

Index(['COD_ALUMNO', 'SEMESTRE', 'TIPO_SEMESTRE', 'CARRERA',
       'SITUACION_MATRICULA', 'CURSOSAPROBADOS', 'CURSOSDESAPROBADOS',
       'CURSOSNSP', 'CREDAPROBADOS', 'CREDDESAPROBADOS',
       ...
       'PROV_TRASLADO_EXTERNO', 'PERIODOS_DESERTO_ULT_SEM_URP',
       'CARRERA_ACTUAL', 'PERIODOS_URP_CARRERA_ACTUAL_REGULAR',
       'PERIODOS_URP_CARRERA_ACTUAL',
       'PROVIENE_TRASLADO_INTERNO_CARRERA_ACTUAL',
       'PRIMER_PERIODO_URP_CARRERA_ACTUAL',
       'ULTIMO_PERIODO_URP_CARRERA_ACTUAL', 'SITUACION_EGRESO', 'TIPO_BECA'],
      dtype='object', length=116)

In [6]:
true_id = [0]
local_id= 0
tt=pd.concat([train,test],sort=False).sort_values('COD_ALUMNO')['SEMESTRE'].values
for i in range(1,len(tt)):
    if tt[i] < tt[i-1]:
        local_id += 1
    true_id.append(local_id)

In [7]:
trte = pd.concat([train,test],sort=False).sort_values('COD_ALUMNO')
trte['true_id']=true_id

In [8]:
train = trte[trte['SITUACION_DEUDA_BOLETA_10'].notna()]
test = trte[trte['SITUACION_DEUDA_BOLETA_10'].isna()]
test.drop(columns=test.columns[test.columns.str.startswith('SITUACION_DEUDA_BOLETA')].values,inplace=True)
train.shape, test.shape

/home/villacorta/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


((256215, 127), (44072, 117))

In [9]:
train.columns

Index(['COD_ALUMNO', 'SEMESTRE', 'TIPO_SEMESTRE', 'CARRERA',
       'SITUACION_MATRICULA', 'CURSOSAPROBADOS', 'CURSOSDESAPROBADOS',
       'CURSOSNSP', 'CREDAPROBADOS', 'CREDDESAPROBADOS',
       ...
       'SITUACION_DEUDA_BOLETA_2', 'SITUACION_DEUDA_BOLETA_3',
       'SITUACION_DEUDA_BOLETA_4', 'SITUACION_DEUDA_BOLETA_5',
       'SITUACION_DEUDA_BOLETA_6', 'SITUACION_DEUDA_BOLETA_7',
       'SITUACION_DEUDA_BOLETA_8', 'SITUACION_DEUDA_BOLETA_9',
       'SITUACION_DEUDA_BOLETA_10', 'true_id'],
      dtype='object', length=127)

In [10]:
test.columns

Index(['COD_ALUMNO', 'SEMESTRE', 'TIPO_SEMESTRE', 'CARRERA',
       'SITUACION_MATRICULA', 'CURSOSAPROBADOS', 'CURSOSDESAPROBADOS',
       'CURSOSNSP', 'CREDAPROBADOS', 'CREDDESAPROBADOS',
       ...
       'PERIODOS_DESERTO_ULT_SEM_URP', 'CARRERA_ACTUAL',
       'PERIODOS_URP_CARRERA_ACTUAL_REGULAR', 'PERIODOS_URP_CARRERA_ACTUAL',
       'PROVIENE_TRASLADO_INTERNO_CARRERA_ACTUAL',
       'PRIMER_PERIODO_URP_CARRERA_ACTUAL',
       'ULTIMO_PERIODO_URP_CARRERA_ACTUAL', 'SITUACION_EGRESO', 'TIPO_BECA',
       'true_id'],
      dtype='object', length=117)

In [11]:
for df in [train, test]:
    #df['TIPO_SEMESTRE'].map({'REGULAR CICLO I':0,'REGULAR CICLO II':1}).astype(int)
    df['PORC_ASISTENCIA']=df['PORC_ASISTENCIA'].replace('SIN ASISTENCIA TOMADA',np.nan).astype(float)
    df['PORC_ASISTENCIA']=df['PORC_ASISTENCIA'].fillna(train['PORC_ASISTENCIA'].mean())
    df['SEXO']=df['SEXO'].map({'M':0,'F':1})
    df['ESTADO_CIVIL']=df['ESTADO_CIVIL'].map({'Conviviente':0,'Soltero(a)':1,'Casado(a)':2}).fillna(0)
    df['SEM_NOTAS_HIST_INICIA_CARRERA'].fillna(0, inplace = True)
    df['SEM_NOTAS_HIST_ULTIMO_CARRERA'].fillna(0, inplace = True)
    
    df['SEM_NOTAS_HIST_INICIA_CARRERA']=df['SEM_NOTAS_HIST_INICIA_CARRERA'].replace('NINGU',0).astype(int)
    df['SEM_NOTAS_HIST_ULTIMO_CARRERA']=df['SEM_NOTAS_HIST_ULTIMO_CARRERA'].replace('NINGU',0).astype(int)
    df['TRASLADO']=(df['CARRERA_ACTUAL']!=df['CARRERA'])
    df['SITUACION_EGRESO']=df['SITUACION_EGRESO'].map({'NO EGRESO':0,'EGRESO':1})

/home/villacorta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/villacorta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/villacorta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [12]:
for df in [train, test]:
    df.drop(columns=['COD_ALUMNO','TIPO_VIVIENDA_DONDE_VIVE','CDEPA_DESCRIPCION','CPROV_DESCRIPCION','CDIST_DESCRIPCION','SEMESTRE'],inplace=True)

In [13]:
y=train[train.columns[train.columns.str.startswith('SITUACION_DEUDA_BOLETA')]].iloc[:,1:].max(axis=1)
X=train[train.columns[~train.columns.str.startswith('SITUACION_DEUDA_BOLETA')]]

In [14]:
te = ce.target_encoder.TargetEncoder(cols=['CARRERA','VIVIENDA_DONDE_VIVE','VIVE_CON',
                                           'ESTADO_CIVIL','TIPO_COLEGIO','CARRERA_ACTUAL',
                                           'SITUACION_EGRESO','TIPO_BECA','TIPO_SEMESTRE'])
X_train = te.fit_transform(X,y)
X_test = te.transform(test)

In [15]:
def xgb_f1(y,t):
    t = t.get_label()
    y_bin = [1. if y_cont > 0.5 else 0. for y_cont in y]
    return 'f1',f1_score(t,y_bin)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)

X = X_train
y = y

predictions=pd.DataFrame()
preds = pd.DataFrame()

f1=[]
i=1
importances=pd.DataFrame()
importances['Features']=X.columns
for a,b in kf.split(X,y):
    X_tr=X.iloc[a,:]
    X_te=X.iloc[b,:]
    y_train=y[a]
    y_test=y[b]
    
    print('---------- Training fold Nº {} ----------'.format(i))
    
    model = xgb.XGBClassifier(n_estimators=10000,nthread=-1,eta=0.2,max_depth=10,subsample=1,
                              colsample_bytree=0.9,colsample_bylevel=0.8,tree_method='hist',
                              feval=xgb_f1,random_state=42,eval_metric=None,
                              verbosity=1,booster='gbtree').fit(X_tr, y_train, eval_set=[(X_te,y_test)],
                                                                early_stopping_rounds=400)
    predictions[str(i)]=model.predict_proba(X_test)[:,1]
    preds[str(i)]=model.predict(X)
    
    i+=1
    gc.collect()

---------- Training fold Nº 1 ----------
[0]	validation_0-error:0.249902
Will train until validation_0-error hasn't improved in 400 rounds.
[1]	validation_0-error:0.244712
[2]	validation_0-error:0.241199
[3]	validation_0-error:0.23995
[4]	validation_0-error:0.238994
[5]	validation_0-error:0.23712
[6]	validation_0-error:0.235189
[7]	validation_0-error:0.234779
[8]	validation_0-error:0.235091
[9]	validation_0-error:0.23474
[10]	validation_0-error:0.234096
[11]	validation_0-error:0.233218
[12]	validation_0-error:0.230759
[13]	validation_0-error:0.229783
[14]	validation_0-error:0.229061
[15]	validation_0-error:0.227734
[16]	validation_0-error:0.227656
[17]	validation_0-error:0.226973
[18]	validation_0-error:0.225665
[19]	validation_0-error:0.225392
[20]	validation_0-error:0.22506
[21]	validation_0-error:0.224417
[22]	validation_0-error:0.224729
[23]	validation_0-error:0.223734
[24]	validation_0-error:0.223734
[25]	validation_0-error:0.223011
[26]	validation_0-error:0.221548
[27]	validation

/home/villacorta/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


---------- Training fold Nº 2 ----------
[0]	validation_0-error:0.250668
Will train until validation_0-error hasn't improved in 400 rounds.
[1]	validation_0-error:0.244131
[2]	validation_0-error:0.241399
[3]	validation_0-error:0.239057
[4]	validation_0-error:0.23855
[5]	validation_0-error:0.237535
[6]	validation_0-error:0.235349
[7]	validation_0-error:0.233007
[8]	validation_0-error:0.232363
[9]	validation_0-error:0.232032
[10]	validation_0-error:0.232441
[11]	validation_0-error:0.231271
[12]	validation_0-error:0.230724
[13]	validation_0-error:0.230295
[14]	validation_0-error:0.229748
[15]	validation_0-error:0.227875
[16]	validation_0-error:0.226489
[17]	validation_0-error:0.225963
[18]	validation_0-error:0.22528
[19]	validation_0-error:0.225084
[20]	validation_0-error:0.224518
[21]	validation_0-error:0.224382
[22]	validation_0-error:0.22366
[23]	validation_0-error:0.223348
[24]	validation_0-error:0.222626
[25]	validation_0-error:0.222255
[26]	validation_0-error:0.221318
[27]	validatio

/home/villacorta/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


---------- Training fold Nº 3 ----------
[0]	validation_0-error:0.248951
Will train until validation_0-error hasn't improved in 400 rounds.
[1]	validation_0-error:0.245399
[2]	validation_0-error:0.242394
[3]	validation_0-error:0.240326
[4]	validation_0-error:0.239057
[5]	validation_0-error:0.23691
[6]	validation_0-error:0.236052
[7]	validation_0-error:0.23492
[8]	validation_0-error:0.233398
[9]	validation_0-error:0.23291
[10]	validation_0-error:0.232441
[11]	validation_0-error:0.231993
[12]	validation_0-error:0.231817
[13]	validation_0-error:0.231798
[14]	validation_0-error:0.231056
[15]	validation_0-error:0.230744
[16]	validation_0-error:0.23049
[17]	validation_0-error:0.229456
[18]	validation_0-error:0.229241
[19]	validation_0-error:0.229085
[20]	validation_0-error:0.228578
[21]	validation_0-error:0.228441
[22]	validation_0-error:0.226724
[23]	validation_0-error:0.226392
[24]	validation_0-error:0.225514
[25]	validation_0-error:0.225065
[26]	validation_0-error:0.224597
[27]	validation

/home/villacorta/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


---------- Training fold Nº 4 ----------
[0]	validation_0-error:0.25221
Will train until validation_0-error hasn't improved in 400 rounds.
[1]	validation_0-error:0.24899
[2]	validation_0-error:0.24579
[3]	validation_0-error:0.242531
[4]	validation_0-error:0.241906
[5]	validation_0-error:0.240911
[6]	validation_0-error:0.237964
[7]	validation_0-error:0.237281
[8]	validation_0-error:0.237047
[9]	validation_0-error:0.236618
[10]	validation_0-error:0.236423
[11]	validation_0-error:0.235954
[12]	validation_0-error:0.235603
[13]	validation_0-error:0.23492
[14]	validation_0-error:0.233281
[15]	validation_0-error:0.232598
[16]	validation_0-error:0.231954
[17]	validation_0-error:0.231427
[18]	validation_0-error:0.230431
[19]	validation_0-error:0.230002
[20]	validation_0-error:0.229202
[21]	validation_0-error:0.228656
[22]	validation_0-error:0.227543
[23]	validation_0-error:0.227212
[24]	validation_0-error:0.226704
[25]	validation_0-error:0.22567
[26]	validation_0-error:0.225689
[27]	validation_

/home/villacorta/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


---------- Training fold Nº 5 ----------
[0]	validation_0-error:0.249698
Will train until validation_0-error hasn't improved in 400 rounds.
[1]	validation_0-error:0.243726
[2]	validation_0-error:0.239823
[3]	validation_0-error:0.237637
[4]	validation_0-error:0.236388
[5]	validation_0-error:0.236017
[6]	validation_0-error:0.232797
[7]	validation_0-error:0.233149
[8]	validation_0-error:0.233305
[9]	validation_0-error:0.232817
[10]	validation_0-error:0.231334
[11]	validation_0-error:0.231021
[12]	validation_0-error:0.230865
[13]	validation_0-error:0.229831
[14]	validation_0-error:0.230182
[15]	validation_0-error:0.229246
[16]	validation_0-error:0.228309
[17]	validation_0-error:0.227879
[18]	validation_0-error:0.225928
[19]	validation_0-error:0.226084
[20]	validation_0-error:0.22585
[21]	validation_0-error:0.225362
[22]	validation_0-error:0.223918
[23]	validation_0-error:0.223371
[24]	validation_0-error:0.222454
[25]	validation_0-error:0.222298
[26]	validation_0-error:0.222005
[27]	validat

In [ ]:
submission=pd.read_csv('../data/data_test.csv',encoding='ISO-8859-1')['COD_ALUMNO'].to_frame()
submission['Morosidad']=predictions.mean(axis=1)
submission.to_csv('../subs/sub04_proba.csv',index=None,encoding='utf-8',sep=',')

In [ ]:
submission